In [ ]:
# import statements
import pymongo
import datetime as dt
import matplotlib.pyplot as plt

from time import sleep
from kafka import KafkaConsumer
from pymongo import MongoClient
from pprint import pprint
from collections import defaultdict

# Connect to the DB
client = MongoClient()
db = client.fit5148_assignment_db
climate_fire = db.climate_fire

# this line is needed for the inline display of graphs in Jupyter Notebook

# %matplotlib inline

In [ ]:
### Task 2.1: Streaming Data Visualisation ###
%matplotlib notebook
topic = "CLIMATE"
plt.rcParams['figure.figsize'] = (20, 8)

def annotate_max(x, y, ax = None):
    ymax = max(y)
    xpos = y.index(ymax)
    xmax = x[xpos]
    text = 'Max: Time={}, Value={}'.format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    ax.annotate(text, xy=(xmax, ymax), xytext=(xmax, ymax+5), arrowprops=dict(facecolor='red', shrink=0.05),)
    
def annotate_min(x, y, ax = None):
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    text = 'Min: Time={}, Value={}'.format(xmin, ymin)
    if not ax:
        ax=plt.gca()
    ax.annotate(text, xy=(xmin, ymin), xytext=(xmin, ymin+5), arrowprops=dict(facecolor='orange', shrink=0.05),)

def connect_kafka_consumer():
    _consumer = None
    try:
         _consumer = KafkaConsumer(topic,
                                   consumer_timeout_ms=60000, # stop iteration if no message after 60 sec
#                                    auto_offset_reset='earliest', # comment this if you don't want to consume earliest available message
                                   bootstrap_servers=['localhost:9092'],
                                   api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _consumer

def init_plots():
    try:
        width = 19
        height = 8
        fig = plt.figure(figsize=(width,height)) # create new figure
        ax = fig.add_subplot(111) # adding the subplot axes to the given grid position
        ax.set_title("Air Temperature against Arrival Time - incl. Max and Min values",pad=15, color="#000000", weight="bold") # giving figure a title
        ax.set_xlabel("Arrival Time",labelpad=15, color="#000000")
        ax.set_ylabel("Air Temperature ($^\circ$C)",labelpad=15, color="#000000")
        ax.yaxis.grid(True, color="#DDDDDD")
        ax.set_ylim(0,50) 
        ax.set_yticks([0,10,20,30,40])
#         fig.show() # displaying the figure
        fig.tight_layout()
        fig.canvas.draw() # drawing on the canvas
        return fig, ax
    except Exception as ex:
        print(str(ex))
    
def consume_messages(consumer, fig, ax):
    try:
        # container for x and y values
        x, y = [], []
        # print('Waiting for messages')
        for message in consumer:
            data = str(message.value.decode('utf-8')).split(', ')
            x.append(data[1][11:])
            y.append(int(data[4]))

            # we start plotting only when we have 5 data points (i.e. after 50 seconds)
            if len(y) > 5:
                ax.clear()
                ax.plot(x, y)
                ax.set_title("Air Temperature against Arrival Time - incl. Max and Min values",pad=15, color="#000000", weight="bold") # giving figure a title
                ax.set_xlabel("Arrival Time",labelpad=15, color="#000000")
                ax.set_ylabel("Air Temperature ($^\circ$C)",labelpad=15, color="#000000")
                ax.yaxis.grid(True, color="#DDDDDD")
                ax.set_ylim(0,50) 
                ax.set_yticks([0,10,20,30,40])
                annotate_max(x,y)
                annotate_min(x,y)
                fig.canvas.draw()
                x.pop(0) # removing the item in the first position
                y.pop(0)
        plt.close('all')
    except Exception as ex:
        print(str(ex))
    
if __name__ == '__main__':
    
    consumer = connect_kafka_consumer()
    fig, ax = init_plots()
    consume_messages(consumer, fig, ax)
    
    